### Random Forest Regressor for environmental factors in a county and cancer rates

In [21]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
# Setting rando seed to get reproducible runs
RSEED = 100

In [22]:
df = pd.read_csv("merged_indicatorsVSCancer.csv")
df

,Unnamed: 0,county,payann,estab,emp,ACID,ENRG,ETOX,EUTR,FOOD,...,MINE,MSW,NREN,OZON,PEST,REN,SMOG,VADD,WATR,cancer
0,22002344,91,128668,594,4954,299.152569,7.286445e+05,1963.026340,26.933851,709.634497,...,0.000000e+00,2149.369209,4.351692e+05,0.002923,3.497395e-02,293475.377416,2949.632736,79914.159702,7851.498268,3455
1,41609466,516,953498,3340,32629,133.420049,6.931874e+03,3765.662479,132.635472,4843.498718,...,1.674254e+04,16148.087370,4.932628e+03,0.007422,8.577729e-02,1999.246044,4438.579174,576096.081165,1633.902304,3455
2,19847485,410,53138,361,1715,102.817249,6.064801e+00,3298.083216,13.598555,126.810329,...,0.000000e+00,554.606997,0.000000e+00,0.000153,4.371549e-02,6.064801,3649.913530,29611.275038,511.844967,3455
3,16221839,469,20778,205,655,30.042694,2.012683e+04,1807.410043,4.819015,76.420984,...,0.000000e+00,309.053357,0.000000e+00,0.000593,2.437340e-02,20126.828138,1064.780004,10666.884789,285.555832,3455
4,23008810,855,61132,455,2161,32.192658,1.131970e+01,416.998063,4.320160,232.490017,...,0.000000e+00,896.650774,0.000000e+00,0.002463,6.501047e-03,11.319704,1165.911030,36511.029249,98.727235,3455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3128,6352509,3774,491651,989,9029,587.935749,3.629382e+06,813.473206,105.117742,1001.624899,...,1.394053e+06,4115.339612,3.628234e+06,0.003711,8.917167e-03,1147.804947,19410.599796,312217.341832,5945.474594,3455
3129,7237066,4524,361056,1402,8689,37.178584,9.249760e+02,7592.166446,25.091516,2044.202199,...,8.200278e+00,6423.591808,4.857347e+02,0.003347,2.019774e-01,439.241290,1412.961358,210925.667302,1777.330068,3455
3130,5811849,3813,145379,442,3287,96.737078,1.919977e+06,112.140420,13.936063,384.841414,...,3.997271e+05,1420.508279,1.919961e+06,0.000773,4.621011e-04,16.608615,3486.862556,97322.166731,582.400056,3455
3131,4442825,3053,31951,283,907,31.417667,2.756565e+05,28.072519,4.183471,101.472042,...,0.000000e+00,518.501787,2.756546e+05,0.000140,3.860924e-07,1.845259,1176.237930,20257.950393,20.344697,3455


In [29]:
data=df[['ACID', 'ENRG',
       'ETOX', 'EUTR', 'FOOD', 'GCC', 'HAPS', 'HAZW', 'HC', 'HNC', 'HRSP',
       'HTOX', 'JOBS', 'LAND', 'METL', 'MINE', 'MSW', 'NREN', 'OZON', 'PEST',
       'REN', 'SMOG', 'VADD', 'WATR', 'cancer']]
data

,ACID,ENRG,ETOX,EUTR,FOOD,GCC,HAPS,HAZW,HC,HNC,...,MINE,MSW,NREN,OZON,PEST,REN,SMOG,VADD,WATR,cancer
0,299.152569,7.286445e+05,1963.026340,26.933851,709.634497,106259.392004,10.732168,15.515155,0.000006,0.000024,...,0.000000e+00,2149.369209,4.351692e+05,0.002923,3.497395e-02,293475.377416,2949.632736,79914.159702,7851.498268,3455
1,133.420049,6.931874e+03,3765.662479,132.635472,4843.498718,43881.445254,31.163349,614.863426,0.000025,0.000110,...,1.674254e+04,16148.087370,4.932628e+03,0.007422,8.577729e-02,1999.246044,4438.579174,576096.081165,1633.902304,3455
2,102.817249,6.064801e+00,3298.083216,13.598555,126.810329,19913.180328,7.238123,3.329358,0.000016,0.000062,...,0.000000e+00,554.606997,0.000000e+00,0.000153,4.371549e-02,6.064801,3649.913530,29611.275038,511.844967,3455
3,30.042694,2.012683e+04,1807.410043,4.819015,76.420984,5751.086704,2.622439,4.793377,0.000005,0.000030,...,0.000000e+00,309.053357,0.000000e+00,0.000593,2.437340e-02,20126.828138,1064.780004,10666.884789,285.555832,3455
4,32.192658,1.131970e+01,416.998063,4.320160,232.490017,6564.143301,3.770756,9.737300,0.000005,0.000021,...,0.000000e+00,896.650774,0.000000e+00,0.002463,6.501047e-03,11.319704,1165.911030,36511.029249,98.727235,3455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3128,587.935749,3.629382e+06,813.473206,105.117742,1001.624899,205214.901809,46.136744,40.714622,0.000113,0.000664,...,1.394053e+06,4115.339612,3.628234e+06,0.003711,8.917167e-03,1147.804947,19410.599796,312217.341832,5945.474594,3455
3129,37.178584,9.249760e+02,7592.166446,25.091516,2044.202199,14614.797183,39.699930,167.219233,0.000015,0.000061,...,8.200278e+00,6423.591808,4.857347e+02,0.003347,2.019774e-01,439.241290,1412.961358,210925.667302,1777.330068,3455
3130,96.737078,1.919977e+06,112.140420,13.936063,384.841414,32123.362015,10.644368,8.324102,0.000019,0.000117,...,3.997271e+05,1420.508279,1.919961e+06,0.000773,4.621011e-04,16.608615,3486.862556,97322.166731,582.400056,3455
3131,31.417667,2.756565e+05,28.072519,4.183471,101.472042,6346.644246,2.766800,2.369265,0.000005,0.000024,...,0.000000e+00,518.501787,2.756546e+05,0.000140,3.860924e-07,1.845259,1176.237930,20257.950393,20.344697,3455


### Splitting to test and train

In [35]:
from sklearn.model_selection import train_test_split
x1=data.loc[:, :'WATR']
scaler = StandardScaler()
X= scaler.fit_transform(x1)
y = data['cancer']
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.3,
                                                    random_state = RSEED)


### Creating Random Forest Model

In [31]:
from sklearn.ensemble import RandomForestRegressor

# Create the model with 10 trees
regressor = RandomForestRegressor(n_estimators = 100,
                                  random_state = RSEED)

# Fit on training data
regressor.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=100, verbose=0, warm_start=False)

### Evaluation

In [36]:
from sklearn import metrics
train_rf_predictions = regressor.predict(X_train)
rf_predictions = regressor.predict(X_test)
MAE_train = metrics.mean_absolute_error(y_train, train_rf_predictions)
RMSE_train = np.sqrt(metrics.mean_squared_error(y_train, train_rf_predictions))
MAPE_train = 100 * np.mean(abs(train_rf_predictions - y_train)/abs(y_train))
accuracy_train = 100 - MAPE_train
r2_train = metrics.r2_score(y_train, train_rf_predictions)

MAE_test = metrics.mean_absolute_error(y_test, rf_predictions)
RMSE_test = np.sqrt(metrics.mean_squared_error(y_test, rf_predictions))
MAPE_test = 100 * np.mean(abs(rf_predictions - y_test)/abs(y_test))
accuracy_test = 100 - MAPE_test
r2_test = metrics.r2_score(y_test, rf_predictions)
print("Evaluation for train:")
print()
#print('Mean Absolute Error Train:', MAE_train)    
#print('Root Mean Squared Error Train:', RMSE_train)
#print('Mean Absolute Percentage Error Train:', MAPE_train)
print('Accuracy Train', str(accuracy_train) + "%")
print('r2_score Train:',r2_train)
print()
print()
print("Evaluation for test:")
print()
#print('Mean Absolute Error Test:', MAE_test)    
#print('Root Mean Squared Error Test:', RMSE_test)
#print('Mean Absolute Percentage Error Test:', MAPE_test)
print('Accuracy Test', str(accuracy_test) + "%")
print('r2_score Test:', r2_test)

Evaluation for train:

Accuracy Train 98.49693160686726%
r2_score Train: 0.8515734547133673


Evaluation for test:

Accuracy Test 95.98601089624304%
r2_score Test: -0.04071041876423509
